# Interaction with the contract

### Web3 instanciation and setup

**NOTE:** Before running this, ensure the blockchain is up by running `deploy.sh`.  

In [5]:
from dotenv import dotenv_values
from functools import wraps
from web3 import Web3
import json

url = "http://127.0.0.1:8545"

# Output from the deploy
contract_address = "0xcf94DD98F88B4975C3bb7074A213140D985FFe57"


env = dotenv_values(".env")
password = env["ACCOUNT_PASSWORD"]

# Also output from deploy
funder_addresses = [
    "0x548AEC5436F39e202205A2f829eeC592E12E7289",
    "0x87834f4A4c629745603e0DC338271246f1798c83",
]
DEFAULT_AMOUNT_FUND = 0.5 # ETH
UNLOCK_DURATION = 300 # Seconds

w3 = Web3(Web3.HTTPProvider(url))

In [6]:
def fund_new_wallet_decorator(func):
    """Wrapper function to automatically send funds from funder adresses to newly created accounts."""

    @wraps(func)
    def wrapper(*args, **kwargs):
        new_wallet_address, new_wallet_private_key = func(*args, **kwargs)

        # Unlock pre-funder wallets
        for address in funder_addresses:
            w3.geth.personal.unlock_account(
                address, password, UNLOCK_DURATION
            )  # Unlock for 5 minutes

        gas_price = w3.eth.gas_price
        gas_limit = w3.eth.estimate_gas(
            {
                "to": new_wallet_address,
                "value": w3.to_wei(DEFAULT_AMOUNT_FUND, "ether"),
                "from": funder_addresses[0],
            }
        )

        for address in funder_addresses:
            transaction = {
                "to": new_wallet_address,
                "value": w3.to_wei(DEFAULT_AMOUNT_FUND, "ether"),
                "gas": gas_limit,
                "gasPrice": gas_price,
                "from": address,
            }
            txn_hash = w3.eth.send_transaction(transaction)
            receipt = w3.eth.wait_for_transaction_receipt(txn_hash)

        return new_wallet_address, new_wallet_private_key

    return wrapper

### Registering an account in the Blockchain

In [7]:
@fund_new_wallet_decorator
def create_new_wallet(password: str):
    """Creates a new wallet in the blockchain"""
    new_wallet = w3.eth.account.create(password)
    new_wallet_address = new_wallet.address
    new_wallet_private_key = new_wallet._private_key.hex()

    return new_wallet_address, new_wallet_private_key


# Usage Example
new_wallet_address, new_wallet_private_key = create_new_wallet("password")

print(f"New Wallet Address: {new_wallet_address}")
print(f"New Wallet Private Key: {new_wallet_private_key}")
print(f"New Wallet Balance: {w3.eth.get_balance(new_wallet_address)}")

New Wallet Address: 0x779072c794C7E3e29A64D03aF9c566448eBb3584
New Wallet Private Key: 0x07bfbe4414032325aa9b689c74d4ac42668b0eba8849ebcb888cc45439ae9d9f
New Wallet Balance: 1000000000000000000


### Interacting with the contract

In [8]:
with open("./build/contracts_Voting_sol_Voting.abi", "r") as f:
    contract_abi = json.load(f)
contract = w3.eth.contract(address=contract_address, abi=contract_abi)

# Election and District info

election_name = "ONI CHAN "
description = "SERJÃO BERRANTEIRO"
# new_wallet_address = "0xc8f52D7E42E61f495aCa36F026cfA6657f0f1c34"
# new_wallet_private_key = "0x47a1495c2842b2891484fe8e17cf4500e836685693a66ec7673d6ea719a651e9"

# Creating an election

tx = contract.functions.createElection(election_name, description).build_transaction(
    {
        "from": new_wallet_address,
        "gas": 2000000,
        "gasPrice": w3.to_wei("20", "gwei"),
        "nonce": w3.eth.get_transaction_count(new_wallet_address),
    }
)

signed_tx = w3.eth.account.sign_transaction(tx, private_key=new_wallet_private_key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print(w3.eth.get_transaction_count(new_wallet_address))

1


In [22]:
contract.functions.getAdminElections("0xc04b62c7A85B3eC327A2e2bA8A93D7f96dDf2E40").call()

[(0,
  'string',
  'string',
  '0xc04b62c7A85B3eC327A2e2bA8A93D7f96dDf2E40',
  '0x0000000000000000000000000000000000000000',
  6,
  52221721488094585246145198526519056873414513544960551659670058848500693598208,
  False,
  1722458927,
  1722458927),
 (0,
  '',
  '',
  '0x0000000000000000000000000000000000000000',
  '0x0000000000000000000000000000000000000000',
  0,
  0,
  False,
  0,
  0)]

In [10]:
contract.functions.getElection(5).call()

ContractLogicError: ('execution reverted: Election does not exist', '0x08c379a000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000017456c656374696f6e20646f6573206e6f74206578697374000000000000000000')

In [11]:
w3.eth.get_transaction("0xc4794ad7a438fb6f87d4ea01436765723bbd8d019b0ef64cc26f0a4b557c0c72")

AttributeDict({'blockHash': HexBytes('0x9792850eb76a979f94b7dd2b35e1b20ff873a28ff01996e6268da60ff30eda29'),
 'blockNumber': 1521,
 'from': '0xc04b62c7A85B3eC327A2e2bA8A93D7f96dDf2E40',
 'gas': 2000000,
 'gasPrice': 20000000000,
 'hash': HexBytes('0xc4794ad7a438fb6f87d4ea01436765723bbd8d019b0ef64cc26f0a4b557c0c72'),
 'input': HexBytes('0x7ad36e84000000000000000000000000000000000000000000000000000000000000004000000000000000000000000000000000000000000000000000000000000000800000000000000000000000000000000000000000000000000000000000000006737472696e6700000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000006737472696e670000000000000000000000000000000000000000000000000000'),
 'nonce': 1,
 'to': '0xcf94DD98F88B4975C3bb7074A213140D985FFe57',
 'transactionIndex': 0,
 'value': 1,
 'type': 0,
 'v': 6948,
 'r': HexBytes('0xb9ba337b1c47566430d2a84ac6ae865b8676239d8203e7d57c811bf4d5f7625a'),
 's': HexBytes('0x65359ba46578c5f0ef9e240d0bd07e90e2

In [ ]:
# tx = contract.functions.addVoter(
#     0, "Leonardo", "12345", new_wallet_address
# ).build_transaction(
#     {
#         "from": new_wallet_address,
#         "gas": 2000000,
#         "gasPrice": w3.to_wei("20", "gwei"),
#         "nonce": w3.eth.get_transaction_count(new_wallet_address),
#     }
# )

# signed_tx = w3.eth.account.sign_transaction(tx, private_key=new_wallet_private_key)
# tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
# tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)